In [1]:
import pandas as pd
from bokeh.plotting import figure, output_file, output_notebook, show, ColumnDataSource, gridplot
from bokeh.models import HoverTool

In [2]:
# Чтобы открыть Excel файл как объект:
# xlsfile = pd.ExcelFile('data/top100.xlsx', header=None)
# dframe = xlsfile.parse(header=None)
# dframe = dframe.rename(columns={30: 'Datetime'})

# Ускоряем загрузку скрипта с помощью преварительно сохранённого в файл DataFrame
# Сохранять с помощью dframe.to_pickle('data/dframe')
dframe = pd.read_pickle('data/dframe')
# Список игроков:
players = dframe[1].unique()

In [3]:
# Готовим данные - добавляем индексы по всем матчам одного игрока и по сотням матчей для одного игрока
list_players = []
for x in range(len(dframe[0].unique())):
    player_dframe = dframe[dframe[1] == players[x]]

    # Отсортируем значения
    player_dframe = player_dframe.sort_values(by='Datetime')

    # Добавим счётчик всех матчей этого игрока
    player_dframe['Сounter'] = [x for x in range(len(player_dframe))]

    # Создаём индекс для группировки
    # TODO: Сделать это красиво и в одну строчку:
    groupper = [item for sublist in [[x for y in range(100)] for x in range(len(player_dframe) // 100)]
                for item in sublist]
    groupper += [groupper[-1]+1 for x in range(len(player_dframe) - len(groupper))]

    # Добавляем в конец столбец с индексом для группировки (индекс 33, можно добавить название столбца)
    # (от SettingWithCopyWarning можно избавиться, установив df.is_copy = True, но я не уверен, что это лучший путь)
    player_dframe[len(player_dframe.columns)+1] = groupper

    # Добавляем запись в список
    list_players.append(player_dframe)

In [4]:
# Данные для оси y
list_unique_heroes = [list_players[x].set_index('Datetime').groupby(33)[4].apply(lambda x: len(x.unique()))
                      for x in range(len(players))]
# Данные для оси x
list_unique_heroes_indexes = list([list_players[x][33].unique() for x in range(0, len(list_players))])

In [5]:
# Рисуем графики:
TOOLS = "pan, wheel_zoom, box_zoom, reset,save, box_select, crosshair"


def plotsomethingnew(plot_number=0, sources=list()):
    # Готовим данные:
    if not isinstance(sources, list):
        sources = []

    foo_nickname = []
    for i in range(0, len(list_unique_heroes[plot_number])):
        foo_nickname.append(dframe[0].unique()[plot_number])

    new_source = ColumnDataSource(
        data=dict(
            x=list_unique_heroes_indexes[plot_number], #list(pandas.core.series.Series)
            y=list_unique_heroes[plot_number],  # pandas.core.series.Series
            nick=foo_nickname,
            unique_heroes=list_unique_heroes[plot_number],
        )
    )

    foo_hover = HoverTool(
            tooltips=[
                ("Index", "$index"),
                ("Nick:", "@nick"),
                ("Unique heroes:", "@unique_heroes"),
                # ("Playing since:", "@playing_since"),
            ]
        )
    sources.append(new_source)

    # Создаём график:
    foo = figure(width=275, height=300, name="foo",
                 title=(str(plot_number) + ' ' + str(foo_nickname[0])), tools=[foo_hover, TOOLS])
    foo.line('x', 'y', source=new_source)
    foo.title_text_font_size = '8pt'
    foo.xaxis.axis_label_text_font_size = '8pt'
    foo.xaxis.major_label_orientation = 0.785  # Pi/4
    # foo.responsive=True
    # адаптивность не работает: http://bokeh.pydata.org/en/0.10.0/docs/user_guide/styling.html#responsive-dimensions
    return foo

In [6]:
# Задаём выходной файл или выводим в IPython Notebook:
# output_file("dota_charts_hundreds.html", title="Выбор героев по сотням матчей")
output_notebook()

BokehJS successfully loaded.

In [7]:
plots = []
plots_row = []
plot_num = 0

# Строим таблицу и помещаем графики в ячейки
# Строим столько графиков, сколько уместится целыми рядами
for x in range(0, (len(players) // 3)):
    plots.append(plots_row)
    for y in range(0, 3):
        plots_row.append(plotsomethingnew(plot_num))
        plot_num += 1
    plots_row = []

grid_layout = gridplot(plots)

In [8]:
show(grid_layout)